# classical search problems 
* 8 queens 
* 8 puzzle 
* snake cube 
* knights tour 

In [82]:
import numpy as np

In [83]:
def seperator(message=""):
    print()
    print("-"*80)
    print(message)
    print("-"*80)
    print()
    

In [84]:
def breath_first_search(root_node):
    """uses a FIFO q """
    count = 0 
    q = [root_node]
#     explored = set()
    while q:
        current = q.pop(0)
        for child in current.get_children():#if no children will return empty list 
            if child.check_goal_state():
                count += 1
            else:
                q.append(child)
#         print(current)
    print("%i goal states found!!"%count)
        

        
    

In [85]:
class Node():
    """subclass to fit your problem"""
    def __init__(self,state):
        self.state = state
    def __repr__(self):
        return str(self.state)
    def check_action_valid(self,action):
        pass 
    def actions(self):
        """returns all valid actions from this state"""
        pass 
    def make_child(self,action):
        """does not check validity of actions, returns new Node obj"""
        pass 
    
    def get_children(self):
        """generator of children of this node"""
        pass 
    def is_goal(self):
        pass 

    

# snake cube 

In [86]:
class Snake_cube_node(Node): 
    #instantiate a class level static variable 
    segments = [3,2,2,2,1,1,1,2,2,1,1,2,1,2,1,1,2]
    cumulate = [0,3,5,7,9,10,11,12,14,16,17,18,20,21,23,24,25]
    count_to_length = dict(zip(cumulate,segments))
    
    action_list = [(value,direction) for value in [1,-1] for direction in range(3)]
    def __init__(self,state,count,loc_last_filled): 
        super().__init__(state)
        self.count = count
        self.loc_last_filled = loc_last_filled
    def check_action_valid(self,action): 
        #action is tiple(val,dir) ex.(-1,2)
        location = np.copy(self.loc_last_filled)
        for _ in range(Snake_cube_node.count_to_length[self.count]): #get number of blocks this segent has 
            location[action[1]] += action[0]
            if not self.check_location_valid(location): 
                return False
        return True
            
        
    def check_location_valid(self,loc): 
        """return false if loc out of bounds or already filled"""
        if all([True  if ((i < 3) and (i >=0)) else False  for i in loc ] ) and self.state[loc[0],loc[1],loc[2]] == 0:
            return True 
        return False 
    def actions(self): 
        return [action for action in Snake_cube_node.action_list  if self.check_action_valid(action)]
    def make_child(self,action): 
        child_state,count_child,location = self.make_child_state(action)
        child_node = Snake_cube_node(child_state,count_child,location)
        return child_node
    
        
    def make_child_state(self,action): 
        #make new copies 
        count_child = self.count 
        child_state = np.copy(self.state)
        location = np.copy(self.loc_last_filled)
        for _ in range(Snake_cube_node.count_to_length[self.count]): #get number of blocks this segent has 
            location[action[1]] += action[0]
            count_child += 1 
            child_state[location[0],location[1],location[2]] = count_child
        return (child_state,count_child,location)
    def get_children(self):
        """generator of children of this node"""
        for action in self.actions(): 
            yield self.make_child(action)
        return []
    def check_goal_state(self):
        if self.count == 27: 
            seperator("GOAL!!!!")
            print(self.state)
            return True
    
    


In [90]:
#run 
start = Snake_cube_node(np.zeros((3,3,3)),0,[-1,0,0])
%time breath_first_search(start)
    


--------------------------------------------------------------------------------
GOAL!!!!
--------------------------------------------------------------------------------

[[[  1.  16.  17.]
  [ 20.  19.  18.]
  [  7.   8.   9.]]

 [[  2.  15.  24.]
  [ 21.  22.  23.]
  [  6.  11.  10.]]

 [[  3.  14.  25.]
  [  4.  13.  26.]
  [  5.  12.  27.]]]

--------------------------------------------------------------------------------
GOAL!!!!
--------------------------------------------------------------------------------

[[[  1.  20.   7.]
  [ 16.  19.   8.]
  [ 17.  18.   9.]]

 [[  2.  21.   6.]
  [ 15.  22.  11.]
  [ 24.  23.  10.]]

 [[  3.   4.   5.]
  [ 14.  13.  12.]
  [ 25.  26.  27.]]]
2 goal states found!!
CPU times: user 56 ms, sys: 4 ms, total: 60 ms
Wall time: 48.1 ms


# 8 queens problem

* total of 92 solutions were found

In [88]:
# define the problem 
class Eight_Queens_Node(Node):
    def __init__(self,state,col):
        super().__init__(state)
        self.col = col #this is the column to be placed next 
        if self.check_goal_state():
            print("-"*80)
            print(self.state)
    def check_action_valid(self,action):
        """check that adding a queen at the location will not cause attack"""
        #check horizontal 
        if self.state[action,:self.col].sum() != 0:
            return False
        for r,c in self.get_location_all_queens():
            if abs(r-action) == abs(c-self.col):
                return False 
        return True
    def get_location_all_queens(self):
        """return (x,y) of all current queens"""
        locations = [(r,c) for r in range(self.state.shape[0]) for c in range(self.state.shape[1]) if self.state[r,c]!=0]
        return locations 
            
    
    def actions(self):
        """returns all valid actions from this state"""
        #if leaf node then no actions 
        return [i for i in range(self.state.shape[0])if self.check_action_valid(i) ]
        
    
    def make_child(self,action):
        """does not check validity of actions, returns new Node obj"""
        child_state = np.copy(self.state)
        child_state[action,self.col] = 1 
        child_node = Eight_Queens_Node(child_state,self.col+1)
        return child_node
    
    def get_children(self):
        """generator of children of this node"""
        for action in self.actions(): 
            yield self.make_child(action)
        return []
            
    def check_goal_state(self):
        return self.col >= self.state.shape[1]

    

In [93]:
#run 
start = Eight_Queens_Node(np.zeros((8,8)),0)
%time breath_first_search(start)


--------------------------------------------------------------------------------
[[ 1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.]]
--------------------------------------------------------------------------------
[[ 1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.]]
--------------------------------------------------------------------------------
[[ 1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  1.  0